In [179]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import random
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import operator

In [128]:
%matplotlib inline

In [129]:
#Create the dataframe we are going to use
df = pd.read_csv("/Users/JustinHolmes/Desktop/Staple Health/County/Mental Health Data/Concatenated_Data/US_County_All_Data_71019.csv", skiprows = 0, sep = ',', header = 0, dtype = {'FIPS': 'Int32'})
#Change any values to get better-looking results
#df['Household Income'] = df['Household Income'].apply(lambda x: x*.001)

#Print the head of the dataframe to see if it imports correctly
#print(df.head())


In [270]:
df2 = df[['FIPS','County_x', 'State_x', "% Female", "% Non-Hispanic White", "% 65 and over", "Household Income (In tens of thousands)", "Percent Educated", 'Population_x', 'Gini Index Estimate', "Mentally Unhealthy Days"]].copy()
df2.rename(columns={'County_x':'County', 'Population_x':'Population', "State_x":"State", "Household Income (In tens of thousands)":"Household Income"}, inplace=True)


In [271]:
df2['Good/Bad Health'] = (df2["Mentally Unhealthy Days"] > 3.9) * 1

In [272]:
#Set the independent variable using the column names
ind1 = 'Gini Index Estimate'

# '% with SNAP'
# 'Average Daily PM2.5 (Air Pollution)'
# 'Association Rate'
# 'Gini Index Estimate'
# '% Adult Obese'
# 'Teen Birth Rate'
# 'Life Expectancy'

In [273]:
#Set the control variables using the column names
ctl1 = '% Female'
ctl2 = '% Non-Hispanic White'
ctl3 = '% 65 and over'
ctl4 = 'Household Income'
ctl5 = 'Percent Educated'

In [274]:
#Set the dependent variable using the column name
dep1 = 'Good/Bad Health'

In [275]:
#Following variables used for the regression analysis
#X = df[[ind1, ind2, ind3, ind4, ind5]].iloc[0:100]
X = df2[[ind1, ctl1, ctl2, ctl3, ctl4, ctl5]]
Y = df2[dep1]
#sm.add_constant(X)

In [276]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=None)
    
logmodel = MLPClassifier(hidden_layer_sizes=(2048,512), solver='adam', max_iter=100000) 
logmodel.fit(X_train, y_train)
y_pred = logmodel.predict(X_test)

df3 = pd.DataFrame({'Predicted': y_pred})

df4 = df2.join(df3,how='right')

df4.head()

,FIPS,County,State,% Female,% Non-Hispanic White,% 65 and over,Household Income,Percent Educated,Population,Gini Index Estimate,Mentally Unhealthy Days,Good/Bad Health,Predicted
0,1001,Autauga,Alabama,51.3,74.5,15.1,58.343,87.700302,55504,45.01,4.3,1,0
1,1003,Baldwin,Alabama,51.5,83.0,19.9,56.607,90.213394,212628,46.18,4.2,1,1
2,1005,Barbour,Alabama,47.2,46.0,18.8,32.490,73.093197,25270,46.22,4.6,1,1
3,1007,Bibb,Alabama,46.5,74.3,16.0,45.795,82.136326,22668,45.18,4.3,1,1
4,1009,Blount,Alabama,50.7,86.9,17.8,48.253,79.784674,58013,43.02,4.7,1,0


In [277]:
err = []
for index, row in df4.iterrows():
    error = row['Predicted']-row['Good/Bad Health']
    if error != 0:
        err.append(1)
print("correct: ", len(df4)-len(err), "out of", len(df4), "predictions")

correct:  317 out of 629 predictions


In [241]:
model = sm.OLS(Y, X, missing = 'drop')
results = model.fit()
print_model = results.summary()  
#print(print_model)

#### Print plot to visualize relationship

In [10]:
#plot = sns.jointplot(x=df[ctl5], y=Y, data=df, kind = 'reg', joint_kws={'line_kws':{'color':'red'}})
#plot